# 🦜🔗 LangChain - Chains & Memory

© Advanced Analytics, Amir Ben Haim, 2024

<br>
<br>
<hr class="dotted">
<br>
<br>

## Setup

<br></br>

### <u>API Keys</u>

In order to use the OpenAI language model, users are required to generate a token.
<br></br>
<u>Follow these simple steps to generate a token with openai:</u>
- Go to <a href="url">https://platform.openai.com/apps</a>  and signup with your email address or connect your Google Account.
- Go to View API Keys on left side of your Personal Account Settings
- Select Create new Secret key
- The API access to OPENAI is a paid service
- You have to set up billing
- You don’t need ChatGPT Plus - The API and ChatGPT subscriptions are billed separately
<br></br>
<p style="background-color:Tomato"> Make sure you read the Pricing information before experimenting</p>
<p style="background-color:Tomato">Once you add your API key, make sure to not share it with anyone! The API key should remain private</p>
<p style="background-color:Tomato">Use the <code>.env</code> file for you API key</p>

<br></br>

### <u>pip install</u>

```powershell
pip install langchain langchain-openai langchain-community
pip install python-dotenv
```

<br></br>

### <u>API Key Setup</u>

Before using LangChain with OpenAI, set your API key:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env

openai_key = os.getenv("OPENAI_API_KEY")
print(openai_key[:5])  # Just to check, don't print the full key!

<br>
<br>
<hr class="dotted">
<br>
<br>

## Initilize SETTINGS

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-4o-2024-11-20")
```

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


llm = ChatOpenAI(model="gpt-4o-2024-11-20")

## 1. Minimal Stateless Chain

```python
response = chain.invoke({"question": "What is LangChain? very short answer"})
print(response.content)
```

In [ ]:
llm = ChatOpenAI(model="gpt-4o-2024-11-20")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{question}")
])
chain = prompt | llm

response = chain.invoke({"question": "What is LangChain? very short answer"})
print(response.content)


## 2. Prompt Template Error

```python
try:
    response = chain.invoke({"xxx": "What is LangChain?"})
    print(response.content)
except...
```

In [ ]:
try:
    response = chain.invoke({"xxx": "What is LangChain?"})
    print(response.content)
except Exception as e:
    print("Error:", e)


## 3. Adding a String Output Parser

```python
response = chain.invoke({"question": "What is LangChain? very short answer"})
print(response)
print(type(response)) 
```

In [ ]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": "What is LangChain? very short answer"})
print(response)
print(type(response)) 

## 4. Using a JSON Output Parser

```python
response = chain.invoke({"question": "Where is Tel Aviv located?"})
print(response)
print(type(response)) 
```

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

prompt_json = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that gives very short answers - you answer the questions in a format of a json, first key as city, second key is country"),
    ("user", "{question}")
])

chain = prompt_json | llm | parser

response = chain.invoke({"question": "Where is Tel Aviv located?"})
print(response)
print(type(response)) 

## 5. Sequential Chain — Two Step Reasoning

```python
def sequential_chain(field):
    subjects = (prompt1 | llm | parser).invoke({"field": field})
    summary = (prompt2 | llm | parser).invoke({"subjects": subjects})
    return summary

print(sequential_chain("Math"))
```

In [ ]:
parser = StrOutputParser()
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "List three major subjects in a certain field {field}, separated by commas."),
    ("user", "{field}")
])
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "Summarize these subjects in one sentence: {subjects}")
])

def sequential_chain(field):
    subjects = (prompt1 | llm | parser).invoke({"field": field})
    summary = (prompt2 | llm | parser).invoke({"subjects": subjects})
    return summary

print(sequential_chain("Math"))


## 6. Chain with a Custom Python Function

Use `sequential_chain()` from Exercise 5, and add to it if necessary
<br></br>

```python
    sequential_chain_with_log('history')
```


In [ ]:
def sequential_chain_with_log(field):
    summary = sequential_chain(field)
    with open(f'{field}.txt', 'w+') as f:
        f.write(summary)
        print(f'a .txt file with summary about {field} has been created')


sequential_chain_with_log('history')

## 7. Basic Stateful Chain with Memory

```python
session_id = "user1"
output1 = chat_chain.invoke({"input": "My name is Sara and I love coffee."}, config={"configurable": {"session_id": session_id}})
print(output1.content)
output2 = chat_chain.invoke({"input": "What is my name and favorite drink?"}, config={"configurable": {"session_id": session_id}})
print(output2.content)
```

In [ ]:
# Prepare LLM and prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that gives very short answers"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])


chain = prompt | llm


store = {}

def get_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]



chat_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
)



session_id = "user1"
output1 = chat_chain.invoke({"input": "My name is Sara and I love coffee."}, config={"configurable": {"session_id": session_id}})
print(output1.content)
output2 = chat_chain.invoke({"input": "What is my name and favorite drink?"}, config={"configurable": {"session_id": session_id}})
print(output2.content)

## 8. Inspect the Chat History

In [ ]:
print(store[session_id].messages)

## 9. Resetting a SPECIFIC Chat Session

Build a `def` function
<br></br>

```python
reset_session(session_id)
print(store[session_id].messages)
print(store)
```

In [ ]:
def reset_session(session_id):
    store[session_id] = ChatMessageHistory()

reset_session(session_id)
print(store[session_id].messages)
print(store)

## 10. Resetting `store`

```python
print(store)
```

In [ ]:
store = {}

print(store)

## 11. Multi-Session Memory

Build a `def` function
<br></br>

```python
session_a = "userA"
chat_chain.invoke({"input": "I am Alex, 25."}, config={"configurable": {"session_id": session_a}})
chat_chain.invoke({"input": "How old am I?"}, config={"configurable": {"session_id": session_a}})

session_b = "userB"
chat_chain.invoke({"input": "I am Dana, 35."}, config={"configurable": {"session_id": session_b}})
chat_chain.invoke({"input": "What is my name?"}, config={"configurable": {"session_id": session_b}})


print_store_chats()
```

In [ ]:
def print_store_chats():
    for k,v in store.items():
        print(f'session_id: {k}')
        print(len(f'session_id: {k})') * '-')
        print(store[k])
        print('\n')




session_a = "userA"
chat_chain.invoke({"input": "I am Alex, 25."}, config={"configurable": {"session_id": session_a}})
chat_chain.invoke({"input": "How old am I?"}, config={"configurable": {"session_id": session_a}})

session_b = "userB"
chat_chain.invoke({"input": "I am Dana, 35."}, config={"configurable": {"session_id": session_b}})
chat_chain.invoke({"input": "What is my name?"}, config={"configurable": {"session_id": session_b}})


print_store_chats()